In [ ]:

# 01: 数据 EDA（JNLI/JSQuAD）—— 会把图和表写入 reports/phase1
import sys, os
repo_root = os.path.abspath(".")
if repo_root not in sys.path: sys.path.append(repo_root)

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from src.data.load_jglue import load_jnli, load_jsquad
OUT = Path("reports/phase1"); OUT.mkdir(parents=True, exist_ok=True)

# JNLI
jnli_tr = load_jnli("train"); jnli_va = load_jnli("validation")
import collections
cnt = collections.Counter(int(x["label"]) for x in jnli_tr)
df = pd.DataFrame([{
    "split":"train","n":len(jnli_tr),
    "lbl0":cnt.get(0,0),"lbl1":cnt.get(1,0),"lbl2":cnt.get(2,0)
}])
cntv = collections.Counter(int(x["label"]) for x in jnli_va)
df = pd.concat([df, pd.DataFrame([{
    "split":"validation","n":len(jnli_va),
    "lbl0":cntv.get(0,0),"lbl1":cntv.get(1,0),"lbl2":cntv.get(2,0)
}])], ignore_index=True)
df.to_csv(OUT/"jnli_stats.csv", index=False)

s1 = [len(x["sentence1"]) for x in jnli_va]
s2 = [len(x["sentence2"]) for x in jnli_va]
plt.figure(); plt.hist(s1, bins=50); plt.title("JNLI sentence1 length (val)"); plt.savefig(OUT/"jnli_s1_len_hist.png"); plt.close()
plt.figure(); plt.hist(s2, bins=50); plt.title("JNLI sentence2 length (val)"); plt.savefig(OUT/"jnli_s2_len_hist.png"); plt.close()

# JSQuAD
js_tr = load_jsquad("train"); js_va = load_jsquad("validation")
clen = [len(x["context"]) for x in js_va]
alen = [len(x["answers"]["text"][0]) if x["answers"]["text"] else 0 for x in js_va]
pd.DataFrame([{
    "split":"train","n":len(js_tr),
    "ctx_len_mean":float(np.mean([len(x["context"]) for x in js_tr])),
    "ans_len_mean":float(np.mean([len(x["answers"]["text"][0]) if x["answers"]["text"] else 0 for x in js_tr])),
}, {
    "split":"validation","n":len(js_va),
    "ctx_len_mean":float(np.mean(clen)),
    "ans_len_mean":float(np.mean(alen)),
}]).to_csv(OUT/"jsquad_stats.csv", index=False)
import matplotlib.pyplot as plt
plt.figure(); plt.hist(clen, bins=60); plt.title("JSQuAD context length (val)"); plt.savefig(OUT/"jsquad_ctx_len_hist.png"); plt.close()
plt.figure(); plt.hist(alen, bins=60); plt.title("JSQuAD answer length (val)"); plt.savefig(OUT/"jsquad_ans_len_hist.png"); plt.close()

print("EDA artifacts written:", list(OUT.iterdir()))
